In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.utils import resample

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
pd.set_option('display.max_columns', None)

https://www.kaggle.com/volodymyrgavrysh/bank-marketing-campaigns-dataset

__Conducted campaigns were based mostly on direct phone calls, offering bank client to place a term deposit.
If after all marking afforts client had agreed to place deposit - target variable marked 'yes', otherwise 'no'__

# Feature Desc

### bank client data:

1. age
    - (numeric)

2. job : type of job 
    - (categorical: "admin.","blue-collar","entrepreneur","housemaid","management","retired","self-employed","services","student","technician","unemployed","unknown")

3. marital : marital status 
    - (categorical: "divorced","married","single","unknown"; note: "divorced" means divorced or widowed)

4. education 
    - (categorical: "basic.4y","basic.6y","basic.9y","high.school","illiterate","professional.course","university.degree","unknown")

5. default: has credit in default? (lalai bayar (?))
    - (categorical: "no","yes","unknown")

6. housing: has housing loan? 
    - (categorical: "no","yes","unknown")

7. loan: has personal loan?
    - (categorical: "no","yes","unknown")
    
### related with the last contact of the current campaign:
8. contact: contact communication type
    - (categorical: "cellular","telephone")
9. month: last contact month of year
    - (categorical: "jan", "feb", "mar", …, "nov", "dec")

10. dayofweek: last contact day of the week
    - (categorical: "mon","tue","wed","thu","fri")

11. duration: last contact duration, in seconds (numeric). ==> (obvious feature (?))
    - __Important note__: this attribute highly affects the output target (e.g., if duration=0 then y="no"). 
    - Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

### other attributes:
12. campaign: number of contacts performed during this campaign and for this client 
    - (numeric, includes last contact)

13. pdays: number of days that passed by after the client was last contacted from a previous campaign
    - (numeric; 999 means client was not previously contacted)

14. previous: number of contacts performed before this campaign and for this client
    - (numeric)

15. poutcome: outcome of the previous marketing campaign 
    - (categorical: "failure","nonexistent","success")

### social and economic context attributes
16. emp.var.rate: employment variation rate - quarterly indicator (???)
    - (numeric)

17. cons.price.idx: consumer price index - monthly indicator
    - changes in the price level of a weighted average __market basket__ of consumer goods and services purchased by households
    - affect inflation
    - (numeric)

18. cons.conf.idx: consumer confidence index - monthly indicator
    - degree of __consumers optimism__ are expressing through their activities of savings and spending.
    - affect consumer behavior
    - (numeric)

19. euribor3m: euribor 3 month rate - daily indicator
    - Euribor (euro interbank offered rate)
    - ??
    - (numeric)

20. nr.employed: number of employees - quarterly indicator
    - Number of employed persons for a quarter. (for the bank ??)
    - (numeric)

### Output variable (desired target):

21. y - has the client subscribed a term deposit?
    - (binary: "yes","no")

In [2]:
df = pd.read_csv('bank-additional-full.csv', sep=';')
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [3]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'duration', 'campaign', 'pdays',
       'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'y'],
      dtype='object')

In [3]:
df.education = df.education.apply(lambda x : x.replace(".", " "))

# Data Description

In [4]:
dfDesc = []
for i in df.columns:
    if df[i].nunique() > 3 :
        bla = df[i].drop_duplicates().sample(3).values
    else :
        bla = df[i].drop_duplicates().values
    
    dfDesc.append([
        i,
        df[i].dtypes,
        df[i].isna().sum(),
        round((((df[i].isna().sum()) / len(df)) * 100), 2),
        df[i].nunique(),
        bla
    ])
    
pd.DataFrame(dfDesc, columns=[
    "Data Features",
    "Data Types",
    "Null",
    "Null Percentage",
    "Unique",
    "Unique Sample"
])

,Data Features,Data Types,Null,Null Percentage,Unique,Unique Sample
0,age,int64,0,0.0,78,"[22, 38, 98]"
1,job,object,0,0.0,12,"[technician, student, blue-collar]"
2,marital,object,0,0.0,4,"[single, unknown, married]"
3,education,object,0,0.0,8,"[unknown, illiterate, basic 9y]"
4,default,object,0,0.0,3,"[no, unknown, yes]"
5,housing,object,0,0.0,3,"[no, yes, unknown]"
6,loan,object,0,0.0,3,"[no, yes, unknown]"
7,contact,object,0,0.0,2,"[telephone, cellular]"
8,month,object,0,0.0,10,"[mar, nov, apr]"
9,day_of_week,object,0,0.0,5,"[thu, mon, tue]"


__Insisght__:
- Tidak ada data null

# Stats Description

In [5]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
age,41188.0,40.024060,10.421250,17.000,32.000,38.000,47.000,98.000
duration,41188.0,258.285010,259.279249,0.000,102.000,180.000,319.000,4918.000
campaign,41188.0,2.567593,2.770014,1.000,1.000,2.000,3.000,56.000
pdays,41188.0,962.475454,186.910907,0.000,999.000,999.000,999.000,999.000
previous,41188.0,0.172963,0.494901,0.000,0.000,0.000,0.000,7.000
emp.var.rate,41188.0,0.081886,1.570960,-3.400,-1.800,1.100,1.400,1.400
cons.price.idx,41188.0,93.575664,0.578840,92.201,93.075,93.749,93.994,94.767
cons.conf.idx,41188.0,-40.502600,4.628198,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,41188.0,3.621291,1.734447,0.634,1.344,4.857,4.961,5.045
nr.employed,41188.0,5167.035911,72.251528,4963.600,5099.100,5191.000,5228.100,5228.100


In [6]:
df.describe(include='O')

,job,marital,education,default,housing,loan,contact,month,day_of_week,poutcome,y
count,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188,41188
unique,12,4,8,3,3,3,2,10,5,3,2
top,admin.,married,university degree,no,yes,no,cellular,may,thu,nonexistent,no
freq,10422,24928,12168,32588,21576,33950,26144,13769,8623,35563,36548


# Cek Imbalance Data dan Pembagian data numerik dan kategorik

In [7]:
df['y'].value_counts()

no     36548
yes     4640
Name: y, dtype: int64

In [8]:
num=['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed']
cat=[i for i in df.columns if i not in num]

# Cek Outliers

In [9]:
Desc_outliers = []

for i in df.describe().columns:
    Q1 = df.describe()[i]['25%']
    Q3 = df.describe()[i]['75%']
    IQR = Q3 - Q1
    upper_fence = Q3 + (1.5*IQR)
    lower_fence = Q1 - (1.5*IQR)
    hitung = 0
    
    for j in df[i]:
        if (j < lower_fence) | (j > upper_fence) :
            hitung+=1
    Desc_outliers.append([
        i,
        hitung,
        upper_fence,
        lower_fence
    ])

out = pd.DataFrame(Desc_outliers, columns=[
    "Data Features",
    "Jumlah Outliers",
    "Upper Fence",
    "Lower Fence"
])
out

,Data Features,Jumlah Outliers,Upper Fence,Lower Fence
0,age,469,69.5000,9.5000
1,duration,2963,644.5000,-223.5000
2,campaign,2406,6.0000,-2.0000
3,pdays,1515,999.0000,999.0000
4,previous,5625,0.0000,0.0000
5,emp.var.rate,0,6.2000,-6.6000
6,cons.price.idx,0,95.3725,91.6965
7,cons.conf.idx,447,-26.9500,-52.1500
8,euribor3m,0,10.3865,-4.0815
9,nr.employed,0,5421.6000,4905.6000


__Insight__:

- Pd kolom age, duration, campaign, pdays, previous dan cons.conf.idx memiliki outliers yg cukup banyak

# ML

### GOALS 

__Nurunin jumlah False Positif__ 

__Naikin Nilai Precision kelas 1 (YES)__

## Data Preprocessing

In [10]:
## Drop kolom yang tidak relevan terkait dengan prediksi model untuk campaign berikutnya
ori = df.copy(deep=True)
df.drop(columns=['duration', 'campaign'], inplace=True)

In [11]:
# mengubah data di kolom target menjadi numerik, Yes ==> 1, No ==> 0
# mengecek persentase sebaran data kolom target

df['y'] = df['y'].map({'yes' : 1, 'no' : 0})
df['y'].value_counts()
round(df['y'].value_counts()/len(df)*100, 2)

0    88.73
1    11.27
Name: y, dtype: float64

In [12]:
# label encoding untuk kolom yg memiliki urutan
df['education'] = df['education'].map({'basic 4y':1
                              , 'basic 6y':2
                              , 'basic 9y' :3
                              , "high school":4
                              , "university degree":5
                              , 'professional course':6
                              , 'illiterate':0
                              , 'unknown':0
                             })

In [13]:
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,1,no,no,no,telephone,may,mon,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,4,unknown,no,no,telephone,may,mon,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
2,37,services,married,4,no,yes,no,telephone,may,mon,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,2,no,no,no,telephone,may,mon,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,4,no,no,yes,telephone,may,mon,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,6,no,yes,no,cellular,nov,fri,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1
41184,46,blue-collar,married,6,no,no,no,cellular,nov,fri,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41185,56,retired,married,5,no,yes,no,cellular,nov,fri,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41186,44,technician,married,6,no,no,no,cellular,nov,fri,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1


## Splitting Data

In [14]:
X = df.drop(columns='y')
y = df['y']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.20, random_state=42)

## Oversampling

In [16]:
df_train = pd.concat([X_train,y_train], axis=1)
non_default = df_train[df_train['y'] == 0] # kelas majority
default = df_train[df_train['y'] == 1] # kelas minority
default_oversample = resample(default, 
                           replace = True, 
                           n_samples = len(non_default),
                           random_state=42)

In [17]:
df_OverSample= pd.concat([non_default, default_oversample])
X_train_OS = df_OverSample.drop(columns='y')
y_train_OS = df_OverSample['y']

## Pipeline

In [18]:
df.columns

Index(['age', 'job', 'marital', 'education', 'default', 'housing', 'loan',
       'contact', 'month', 'day_of_week', 'pdays', 'previous', 'poutcome',
       'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m',
       'nr.employed', 'y'],
      dtype='object')

In [24]:
num_columns = ['pdays', 'previous', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed', 'age', 'education']
cat_columns = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'month', 'day_of_week', 'poutcome']

In [25]:
numeric_pipeline = Pipeline([
    ('scaler', RobustScaler())
])

categoric_pipeline = Pipeline([
    ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer([
    ('numeric', numeric_pipeline, num_columns),
    ('categorical', categoric_pipeline, cat_columns)
])

pipe_DT = Pipeline([
    ('prep', preprocessor),
    ('algo', DecisionTreeClassifier())
])

pipe_KNN = Pipeline([
    ('prep', preprocessor),
    ('algo', KNeighborsClassifier())
])

## Def Eva Metrix

Fungsi def yg isinya evaluation metrix.
Evaluation matrix yang dipakai kali ini adalah Cunfusion matrix dan Classification report

In [26]:
def conf_mat (Model, X_train, X_test, y_train, y_test,Nama):
    y_pred_test = Model.predict(X_test)
    cm_test = confusion_matrix(y_test, y_pred_test, labels=[1,0])
    df = pd.DataFrame(cm_test, index = ['Akt1', 'Akt0'], columns=['Pred1', 'Pred0'])
    print( 'Classification report data TEST ' + Nama + '\n\n', classification_report(y_test, y_pred_test))
    
    y_pred_train = Model.predict(X_train)
    print( 'Classification report data TRAIN ' + Nama + '\n\n', classification_report(y_train, y_pred_train))
#     return plt.title('Confusion matrix data test ' + Nama + '\n'), sns.heatmap(df, annot=True)
    print ('Confusion matrix data test ' + Nama + '\n')
    return df

In [27]:
def prec_rec (Model, X_test, y_test, Nama):
    data = {}
    prec = []
    rec = []
    # for i in Model :
    y_pred_ts = Model.predict(X_test)
    precision = precision_score(y_test, y_pred_ts)
    recall = recall_score(y_test, y_pred_ts)
    prec.append(precision)
    rec.append(recall)
    data[Nama] = [prec[0], rec[0]]
    
    df = pd.DataFrame(data, index=['Precison', 'Recall'])
    return df

## Model Fitting

In [28]:
pipe_DT.fit(X_train_OS, y_train_OS)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  ['pdays', 'previous',
                                                   'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed', 'age',
                                                   'education']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['job', 'marital', 'default',
             

In [29]:
conf_mat(pipe_DT, X_train_OS, X_test, y_train_OS, y_test, 'DT Over Sampling')

Classification report data TEST DT Over Sampling

               precision    recall  f1-score   support

           0       0.92      0.90      0.91      7310
           1       0.31      0.35      0.33       928

    accuracy                           0.84      8238
   macro avg       0.61      0.62      0.62      8238
weighted avg       0.85      0.84      0.84      8238

Classification report data TRAIN DT Over Sampling

               precision    recall  f1-score   support

           0       1.00      0.99      0.99     29238
           1       0.99      1.00      0.99     29238

    accuracy                           0.99     58476
   macro avg       0.99      0.99      0.99     58476
weighted avg       0.99      0.99      0.99     58476

Confusion matrix data test DT Over Sampling



,Pred1,Pred0
Akt1,325,603
Akt0,735,6575


In [30]:
pipe_KNN.fit(X_train_OS, y_train_OS)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  ['pdays', 'previous',
                                                   'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed', 'age',
                                                   'education']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['job', 'marital', 'default',
             

In [31]:
conf_mat(pipe_KNN, X_train_OS, X_test, y_train_OS, y_test, 'KNN Over Sampling')

Classification report data TEST KNN Over Sampling

               precision    recall  f1-score   support

           0       0.94      0.79      0.86      7310
           1       0.26      0.58      0.36       928

    accuracy                           0.77      8238
   macro avg       0.60      0.69      0.61      8238
weighted avg       0.86      0.77      0.80      8238

Classification report data TRAIN KNN Over Sampling

               precision    recall  f1-score   support

           0       1.00      0.85      0.92     29238
           1       0.87      1.00      0.93     29238

    accuracy                           0.92     58476
   macro avg       0.93      0.92      0.92     58476
weighted avg       0.93      0.92      0.92     58476

Confusion matrix data test KNN Over Sampling



,Pred1,Pred0
Akt1,540,388
Akt0,1500,5810


_____________

In [32]:
X_train_OS

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
25611,49,blue-collar,married,3,unknown,no,no,cellular,nov,wed,999,0,nonexistent,-0.1,93.200,-42.0,4.120,5195.8
26010,37,entrepreneur,married,5,no,no,no,telephone,nov,wed,999,1,failure,-0.1,93.200,-42.0,4.120,5195.8
297,36,admin.,married,5,no,yes,no,telephone,may,mon,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0
36344,59,retired,divorced,5,no,no,no,cellular,jun,tue,999,0,nonexistent,-2.9,92.963,-40.8,1.262,5076.2
21806,29,admin.,single,5,no,no,no,cellular,aug,wed,999,0,nonexistent,1.4,93.444,-36.1,4.964,5228.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12425,31,entrepreneur,married,2,no,yes,yes,cellular,jul,mon,999,0,nonexistent,1.4,93.918,-42.7,4.960,5228.1
38689,26,admin.,single,4,no,yes,no,cellular,nov,mon,3,1,success,-3.4,92.649,-30.1,0.715,5017.5
34610,27,blue-collar,single,3,no,yes,yes,cellular,may,thu,999,0,nonexistent,-1.8,92.893,-46.2,1.266,5099.1
37465,29,admin.,single,5,no,yes,yes,cellular,aug,thu,3,1,success,-2.9,92.201,-31.4,0.873,5076.2


## HYPERPARAMETER TUNING

In [33]:
skf = StratifiedKFold(n_splits=3)

========================

__DT__

========================

In [34]:
# pipe_DT.get_params()

In [35]:
param_DT = {
    'algo__max_depth': list(np.arange(0, 101, 10)) + [None],
    'algo__min_samples_leaf': np.arange(1, 101, 10),
    'algo__min_samples_split': np.arange(1, 101, 10)
}

In [36]:
bestparam_DT = {
    'algo__max_depth': [75],
    'algo__min_samples_leaf': [1],
    'algo__max_features': [0.6]
    # 'algo__class_weight': [None, {0:.4, 1:.6}, {0:.3, 1:.7}, {0:.2, 1:.8}, {0:.15, 1: .85}, {0:.1, 1:.9}]
}

In [37]:
DT_GS = GridSearchCV(pipe_DT, param_DT, cv=skf, n_jobs=-1, verbose=1, scoring='precision')

In [38]:
DT_GS.fit(X_train_OS, y_train_OS)

Fitting 3 folds for each of 1200 candidates, totalling 3600 fits


GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('prep',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('scaler',
                                                                                          RobustScaler())]),
                                                                         ['pdays',
                                                                          'previous',
                                                                          'emp.var.rate',
                                                                          'cons.price.idx',
                                                                          'cons.conf.idx',
                                                                          'euribor3m',
                                                   

In [39]:
DT_GS.best_params_

{'algo__max_depth': 50,
 'algo__min_samples_leaf': 1,
 'algo__min_samples_split': 11}

In [40]:
pd.DataFrame(DT_GS.cv_results_).columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_algo__max_depth', 'param_algo__min_samples_leaf',
       'param_algo__min_samples_split', 'params', 'split0_test_score',
       'split1_test_score', 'split2_test_score', 'mean_test_score',
       'std_test_score', 'rank_test_score'],
      dtype='object')

______________
_UNTUK yng best estimator_
_____________

In [41]:
DT_Tune = DT_GS.best_estimator_

In [42]:
DT_Tune.fit(X_train_OS, y_train_OS)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('numeric',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  ['pdays', 'previous',
                                                   'emp.var.rate',
                                                   'cons.price.idx',
                                                   'cons.conf.idx', 'euribor3m',
                                                   'nr.employed', 'age',
                                                   'education']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['job', 'marital', 'default',
             

In [43]:
conf_mat(DT_Tune, X_train_OS, X_test, y_train_OS, y_test, 'DT Tuning best estimator')

Classification report data TEST DT Tuning best estimator

               precision    recall  f1-score   support

           0       0.92      0.88      0.90      7310
           1       0.29      0.39      0.33       928

    accuracy                           0.82      8238
   macro avg       0.60      0.63      0.61      8238
weighted avg       0.85      0.82      0.83      8238

Classification report data TRAIN DT Tuning best estimator

               precision    recall  f1-score   support

           0       0.99      0.95      0.97     29238
           1       0.96      0.99      0.97     29238

    accuracy                           0.97     58476
   macro avg       0.97      0.97      0.97     58476
weighted avg       0.97      0.97      0.97     58476

Confusion matrix data test DT Tuning best estimator



,Pred1,Pred0
Akt1,360,568
Akt0,891,6419


In [48]:
param_DT = {
    'algo__max_depth': np.arange(40, 61, 2),
    'algo__min_samples_leaf': np.arange(1, 11, 2),
    'algo__min_samples_split': np.arange(1, 21, 2)
}

In [49]:
DT_GS = GridSearchCV(pipe_DT, param_DT, cv=skf, n_jobs=-1, verbose=1, scoring='precision')
DT_GS.fit(X_train_OS, y_train_OS)
DT_GS.best_params_

Fitting 3 folds for each of 550 candidates, totalling 1650 fits


{'algo__max_depth': 50,
 'algo__min_samples_leaf': 1,
 'algo__min_samples_split': 3}

In [50]:
DT_Tune2 = DT_GS.best_estimator_
DT_Tune2.fit(X_train_OS, y_train_OS)
conf_mat(DT_Tune2, X_train_OS, X_test, y_train_OS, y_test, 'DT Tuning 2 best estimator')

Classification report data TEST DT Tuning 2 best estimator

               precision    recall  f1-score   support

           0       0.92      0.90      0.91      7310
           1       0.31      0.35      0.33       928

    accuracy                           0.84      8238
   macro avg       0.61      0.62      0.62      8238
weighted avg       0.85      0.84      0.84      8238

Classification report data TRAIN DT Tuning 2 best estimator

               precision    recall  f1-score   support

           0       1.00      0.99      0.99     29238
           1       0.99      1.00      0.99     29238

    accuracy                           0.99     58476
   macro avg       0.99      0.99      0.99     58476
weighted avg       0.99      0.99      0.99     58476

Confusion matrix data test DT Tuning 2 best estimator



,Pred1,Pred0
Akt1,323,605
Akt0,730,6580


In [4]:
df.to_html('data.html')